In [1]:
import numpy as np
import math
import matplotlib.pyplot as plt
from numba import jit
import pandas as pd
import warnings
warnings.filterwarnings("ignore")
plt.rcParams["font.sans-serif"] = ["SimHei"]
plt.rcParams["axes.unicode_minus"] = False

In [13]:
#定义几何布朗运动函数
def mcs_simulation(S0,r,b,T,sigma,paths,steps):
    """
    模拟几何布朗运动
    S0: 初始价格
    r: 无风险利率
    b:  持有成本
    T: 时间长度
    sigma: 波动率
    paths: 模拟路径数量
    steps: 时间步长数量
    """
    dt = T/steps
    S_path = np.zeros((steps+1,paths))
    S_path[0] = S0
    for step in range(1,steps+1):
        rn = np.random.standard_normal(paths)
        S_path[step] = S_path[step-1] * np.exp((b-sigma**2/2)*dt + sigma*np.sqrt(dt)*rn)
    return S_path

#计算每条路径敲入敲出情况，并求期望现金流，得到期权价值
@jit(forceobj=True)
def auto_callable(S0,r,b,T,sigma,paths,steps,knock_out,knock_in,coupon):
    """
    模拟自动可调用期权
    S0: 初始价格
    r: 无风险利率
    b:  持有成本
    T: 时间长度
    sigma: 波动率
    paths: 模拟路径数量
    steps: 时间步长数量
    knock_out: 敲出价格
    knock_in: 敲入价格
    coupon: 票息
    """
    S_paths = mcs_simulation(S0,r,b,T,sigma,paths,steps)
    cash_flow = np.zeros(paths) #用于存储现金流

    knock_out_count = 0
    knock_in_count = 0
    not_in_out = 0
    for i in range(paths):#对每条模拟路径进行循环，分析敲入敲出情况
        knock_out_index = np.where(S_paths[:,i] >= knock_out*S0)[0]
        knock_out_index = knock_out_index[knock_out_index % 21 ==0] # 简化假设每21个交易日为观察日
        knock_in_index = np.where(S_paths[:,i] < knock_in*S0)[0] #以表现差的确定敲入，相当于有一个敲入就算敲入

        #1.敲出
        if knock_out_index.size > 0:
            t = knock_out_index[0]
            cash_flow[i] = coupon*t/252*np.exp(-r*t/252)
            knock_out_count = knock_out_count + 1
        #2.未敲出也未敲入
        elif knock_out_index.size == 0 and knock_in_index.size ==0:
            cash_flow[i]= coupon*T*np.exp(-r*T)
            not_in_out = not_in_out + 1
        #敲入但未敲出
        elif knock_out_index.size == 0 and knock_in_index.size>0:
            payoff = min(S_paths[-1][i]/S0-1,0)
            cash_flow[i] = payoff*np.exp(-r*T)
            knock_in_count = knock_in_count + 1
        else:
            pass
    seneriao_count = {"敲出比率":knock_out_count/paths,"敲入比率":knock_in_count/paths,"未敲入敲出":not_in_out/paths}
    return cash_flow.mean(),seneriao_count


In [14]:
%time auto_callable(S0=100,r=0.03,b=0.03,T=1,sigma=0.13,paths=500000,steps = 252,knock_out=1.03,knock_in=0.85,coupon=0.2)


CPU times: total: 2.75 s
Wall time: 10.4 s


(0.05040423384830282, {'敲出比率': 0.73787, '敲入比率': 0.12685, '未敲入敲出': 0.13528})